In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
! pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 1.9 MB/s eta 0:00:00


In [3]:
import keras_tuner as kt

In [4]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [6]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [10]:
tuner.search(img_train, label_train, epochs=10, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 23s]
val_accuracy: 0.8918333053588867

Best val_accuracy So Far: 0.8918333053588867
Total elapsed time: 00h 24m 06s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 160 and the optimal learning rate for the optimizer
is 0.001.



In [11]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.5135 - accuracy: 0.8209 - val_loss: 0.3972 - val_accuracy: 0.8587
Epoch 2/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.3859 - accuracy: 0.8611 - val_loss: 0.4028 - val_accuracy: 0.8531
Epoch 3/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3485 - accuracy: 0.8721 - val_loss: 0.3674 - val_accuracy: 0.8612
Epoch 4/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3186 - accuracy: 0.8821 - val_loss: 0.3510 - val_accuracy: 0.8741
Epoch 5/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3022 - accuracy: 0.8888 - val_loss: 0.3308 - val_accuracy: 0.8806
Epoch 6/50
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2861 - accuracy: 0.8936 - val_loss: 0.3234 - val_accuracy: 0.8823
Epoch 7/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2740 - accuracy: 0.8980 - val_loss: 0.3310 - val_accuracy:

In [12]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/21
1500/1500 [==============================] - 8s 5ms/step - loss: 0.5106 - accuracy: 0.8206 - val_loss: 0.4176 - val_accuracy: 0.8511
Epoch 2/21
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3833 - accuracy: 0.8628 - val_loss: 0.3647 - val_accuracy: 0.8695
Epoch 3/21
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3415 - accuracy: 0.8751 - val_loss: 0.3480 - val_accuracy: 0.8762
Epoch 4/21
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3166 - accuracy: 0.8819 - val_loss: 0.3440 - val_accuracy: 0.8785
Epoch 5/21
1500/1500 [==============================] - 7s 4ms/step - loss: 0.2968 - accuracy: 0.8903 - val_loss: 0.3347 - val_accuracy: 0.8798
Epoch 6/21
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2827 - accuracy: 0.8953 - val_loss: 0.3303 - val_accuracy: 0.8819
Epoch 7/21
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2657 - accuracy: 0.9009 - val_loss: 0.3292 - val_accuracy:

In [13]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.3597 - accuracy: 0.8914
[test loss, test accuracy]: [0.35967132449150085, 0.8913999795913696]
